In [29]:
#Classifier imports
import pandas as pd
import numpy as np
import sys
import pickle as pkl
import pickle
import joblib
import itertools
from math import ceil 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC

# Performance metrics
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error)


In [4]:
# load  preprocessed dataset
data=pd.read_csv("train.csv")

#Split data into training data (X) and target label(Y)
#convert data from dataframe to matrix
data_new=data.values

X=data_new[:,:-1] #gets elements of array but the last one
Y=data_new[:,-1]

#train-test split
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split( X, Y, random_state=7,test_size=0.50)


In [5]:
#print the shape of training and test splits .50 
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(523557, 20)
(523558, 20)
(523557,)
(523558,)


In [6]:
# Initialize our classifiers
LSVC = LinearSVC(C=10,loss='hinge')
LR = LogisticRegression(C=1.0,solver='newton-cg')


In [11]:
from datetime import datetime
start=datetime.now()
LR.fit(x_train,y_train)
stop=datetime.now()
#Execution time of the model
execution_time_LR = stop-start
print(execution_time_LR)
y_test_LR_model = LR.predict(x_test)
print("LR Accuracy :", accuracy_score(y_test, y_test_LR_model))
pickle.dump(LR, open("LR.dat", "wb"))

E:\ProgramData\Anaconda3\envs\tf_gpu\lib\site-packages\scipy\optimize\linesearch.py:466: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
E:\ProgramData\Anaconda3\envs\tf_gpu\lib\site-packages\scipy\optimize\linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)


0:00:39.382509
LR Accuracy : 0.9303954862689521


E:\ProgramData\Anaconda3\envs\tf_gpu\lib\site-packages\sklearn\utils\optimize.py:212: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)


In [12]:
start1=datetime.now()
LSVC.fit(x_train,y_train)
stop1=datetime.now()
#Execution time of the model
execution_time_LSVC = stop1-start1
print(execution_time_LSVC)

y_test_LSVC_model = LSVC.predict(x_test)
print("LSVC Accuracy :", accuracy_score(y_test, y_test_LSVC_model))
pickle.dump(LSVC, open("LSVC.dat", "wb"))

0:03:06.360962
LSVC Accuracy : 0.7559047899182134


E:\ProgramData\Anaconda3\envs\tf_gpu\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [23]:
######### Training Light GBM Model#######
import lightgbm as lgb
import datetime
from pandas import Series, DataFrame 

param = {'num_leaves':128,'learning_rate':0.1,'n_estimators':64}
param['metric'] = ['auc', 'binary_logloss']


In [24]:
#create train data object for lgbm
#label is used to define our outcome variable
train_data=lgb.Dataset(x_train,label=y_train)

#training our model using light gbm
from datetime import datetime
start2=datetime.now()
lgbm=lgb.train(param,train_data)
stop2=datetime.now()
#Execution time of the model
execution_time_lgbm = stop2-start2
print(execution_time_lgbm)

0:00:02.022744


In [26]:
y_test_lgbm = lgbm.predict(x_test)
y_test_lgbm[0:10]


array([4.36069695e-04, 9.99151890e-01, 9.99151890e-01, 9.33949674e-04,
       5.08677538e-04, 9.99151890e-01, 4.72591443e-04, 9.99151890e-01,
       4.18369279e-04, 5.08677538e-04])

In [38]:
#calculating accuracy of lgbm
print("LGBM Accuracy :", accuracy_score(y_test, y_test_lgbm.round()))
from sklearn.metrics import roc_auc_score
auc_lgbm = roc_auc_score(y_test,y_test_lgbm)
print("LGBM AUC score :",auc_lgbm)


LGBM Accuracy : 0.9991443163890151
LGBM AUC score : 0.9999202244408073


In [39]:
###save lightgbm model###
pickle.dump(lgbm, open("LGBM.dat", "wb"))

In [41]:
######### Training Catboost Model#######
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
CB_model = CatBoostClassifier(iterations=512,learning_rate=0.1,depth=16, loss_function= 'Logloss', eval_metric='AUC',use_best_model=True,random_seed=42)
from datetime import datetime
start3=datetime.now()
CB_model.fit(x_train,y_train, eval_set=(x_test,y_test))
stop3=datetime.now()
#Execution time of the model
execution_time_CB = stop3-start3
print(execution_time_CB)



0:	test: 0.9987700	best: 0.9987700 (0)	total: 5.85s	remaining: 49m 50s
1:	test: 0.9994033	best: 0.9994033 (1)	total: 11.7s	remaining: 49m 50s
2:	test: 0.9995872	best: 0.9995872 (2)	total: 17.6s	remaining: 49m 48s
3:	test: 0.9997120	best: 0.9997120 (3)	total: 23.5s	remaining: 49m 49s
4:	test: 0.9997341	best: 0.9997341 (4)	total: 29.5s	remaining: 49m 49s
5:	test: 0.9997932	best: 0.9997932 (5)	total: 35.5s	remaining: 49m 57s
6:	test: 0.9998381	best: 0.9998381 (6)	total: 41.4s	remaining: 49m 43s
7:	test: 0.9998505	best: 0.9998505 (7)	total: 47.6s	remaining: 49m 56s
8:	test: 0.9998583	best: 0.9998583 (8)	total: 54.5s	remaining: 50m 45s
9:	test: 0.9998633	best: 0.9998633 (9)	total: 1m	remaining: 50m 47s
10:	test: 0.9999125	best: 0.9999125 (10)	total: 1m 6s	remaining: 50m 39s
11:	test: 0.9999222	best: 0.9999222 (11)	total: 1m 12s	remaining: 50m 31s
12:	test: 0.9999277	best: 0.9999277 (12)	total: 1m 18s	remaining: 50m 24s
13:	test: 0.9999345	best: 0.9999345 (13)	total: 1m 24s	remaining: 50m 17

112:	test: 0.9999876	best: 0.9999876 (112)	total: 10m 56s	remaining: 38m 39s
113:	test: 0.9999875	best: 0.9999876 (112)	total: 11m 1s	remaining: 38m 30s
114:	test: 0.9999875	best: 0.9999876 (112)	total: 11m 1s	remaining: 38m 5s
115:	test: 0.9999875	best: 0.9999876 (112)	total: 11m 8s	remaining: 38m
116:	test: 0.9999876	best: 0.9999876 (116)	total: 11m 14s	remaining: 37m 55s
117:	test: 0.9999876	best: 0.9999876 (117)	total: 11m 20s	remaining: 37m 50s
118:	test: 0.9999877	best: 0.9999877 (118)	total: 11m 25s	remaining: 37m 44s
119:	test: 0.9999876	best: 0.9999877 (118)	total: 11m 31s	remaining: 37m 39s
120:	test: 0.9999877	best: 0.9999877 (120)	total: 11m 36s	remaining: 37m 31s
121:	test: 0.9999877	best: 0.9999877 (120)	total: 11m 41s	remaining: 37m 22s
122:	test: 0.9999877	best: 0.9999877 (122)	total: 11m 46s	remaining: 37m 14s
123:	test: 0.9999877	best: 0.9999877 (123)	total: 11m 51s	remaining: 37m 5s
124:	test: 0.9999879	best: 0.9999879 (124)	total: 11m 57s	remaining: 37m
125:	test: 0

220:	test: 0.9999897	best: 0.9999897 (188)	total: 18m 45s	remaining: 24m 42s
221:	test: 0.9999897	best: 0.9999897 (188)	total: 18m 47s	remaining: 24m 33s
222:	test: 0.9999897	best: 0.9999897 (188)	total: 18m 50s	remaining: 24m 24s
223:	test: 0.9999897	best: 0.9999897 (188)	total: 18m 52s	remaining: 24m 16s
224:	test: 0.9999897	best: 0.9999897 (188)	total: 18m 55s	remaining: 24m 7s
225:	test: 0.9999897	best: 0.9999897 (188)	total: 18m 57s	remaining: 23m 59s
226:	test: 0.9999897	best: 0.9999897 (188)	total: 18m 59s	remaining: 23m 51s
227:	test: 0.9999897	best: 0.9999897 (188)	total: 19m 2s	remaining: 23m 42s
228:	test: 0.9999897	best: 0.9999897 (188)	total: 19m 6s	remaining: 23m 37s
229:	test: 0.9999897	best: 0.9999897 (188)	total: 19m 9s	remaining: 23m 29s
230:	test: 0.9999897	best: 0.9999897 (188)	total: 19m 13s	remaining: 23m 23s
231:	test: 0.9999897	best: 0.9999897 (188)	total: 19m 16s	remaining: 23m 15s
232:	test: 0.9999897	best: 0.9999897 (188)	total: 19m 18s	remaining: 23m 7s
233:

327:	test: 0.9999897	best: 0.9999897 (188)	total: 24m 50s	remaining: 13m 55s
328:	test: 0.9999897	best: 0.9999897 (188)	total: 24m 53s	remaining: 13m 50s
329:	test: 0.9999897	best: 0.9999897 (188)	total: 24m 56s	remaining: 13m 45s
330:	test: 0.9999897	best: 0.9999897 (188)	total: 24m 59s	remaining: 13m 40s
331:	test: 0.9999897	best: 0.9999897 (188)	total: 25m 3s	remaining: 13m 34s
332:	test: 0.9999897	best: 0.9999897 (188)	total: 25m 6s	remaining: 13m 29s
333:	test: 0.9999897	best: 0.9999897 (188)	total: 25m 9s	remaining: 13m 24s
334:	test: 0.9999897	best: 0.9999897 (188)	total: 25m 12s	remaining: 13m 19s
335:	test: 0.9999897	best: 0.9999897 (188)	total: 25m 16s	remaining: 13m 14s
336:	test: 0.9999897	best: 0.9999897 (188)	total: 25m 19s	remaining: 13m 8s
337:	test: 0.9999897	best: 0.9999897 (188)	total: 25m 22s	remaining: 13m 3s
338:	test: 0.9999897	best: 0.9999897 (188)	total: 25m 25s	remaining: 12m 58s
339:	test: 0.9999897	best: 0.9999897 (188)	total: 25m 28s	remaining: 12m 53s
340:

435:	test: 0.9999897	best: 0.9999897 (188)	total: 30m 33s	remaining: 5m 19s
436:	test: 0.9999897	best: 0.9999897 (188)	total: 30m 36s	remaining: 5m 15s
437:	test: 0.9999897	best: 0.9999897 (188)	total: 30m 39s	remaining: 5m 10s
438:	test: 0.9999897	best: 0.9999897 (188)	total: 30m 43s	remaining: 5m 6s
439:	test: 0.9999897	best: 0.9999897 (188)	total: 30m 46s	remaining: 5m 2s
440:	test: 0.9999897	best: 0.9999897 (188)	total: 30m 49s	remaining: 4m 57s
441:	test: 0.9999897	best: 0.9999897 (188)	total: 30m 52s	remaining: 4m 53s
442:	test: 0.9999897	best: 0.9999897 (188)	total: 30m 56s	remaining: 4m 49s
443:	test: 0.9999897	best: 0.9999897 (188)	total: 30m 59s	remaining: 4m 44s
444:	test: 0.9999897	best: 0.9999897 (188)	total: 31m 2s	remaining: 4m 40s
445:	test: 0.9999897	best: 0.9999897 (188)	total: 31m 5s	remaining: 4m 36s
446:	test: 0.9999897	best: 0.9999897 (188)	total: 31m 8s	remaining: 4m 31s
447:	test: 0.9999897	best: 0.9999897 (188)	total: 31m 12s	remaining: 4m 27s
448:	test: 0.9999

In [42]:
###Calculate validation accuracy###
CB_prediction = CB_model.predict(x_test)
print("CB Accuracy :", accuracy_score(y_test, CB_prediction))

CB Accuracy : 0.9995014878962789


In [43]:
from sklearn.metrics import roc_auc_score
auc_CB = roc_auc_score(y_test,CB_prediction)
print("CatBoost AUC score :",auc_CB)

CatBoost AUC score : 0.9994748406507801


In [51]:
CB_pred_proba = CB_model.predict_proba(x_test)
CB_preds= CB_pred_proba[:,1][0:10]
print(CB_preds)

[1.22089869e-05 9.99999308e-01 9.99988586e-01 3.70357469e-06
 3.08379119e-07 9.99998265e-01 4.87092546e-07 9.99999622e-01
 6.34749101e-06 3.84610797e-06]


In [45]:
# Get predictions
CB_prediction = CB_model.predict(x_test)
print(CB_prediction[0:10])
# Get predicted RawFormulaVal
CB_preds_raw = CB_model.predict(x_test,prediction_type='RawFormulaVal')
print(CB_preds_raw[0:10])

[0. 1. 1. 0. 0. 1. 0. 1. 0. 0.]
[-11.31332604  14.18402759  11.38066688 -12.50620837 -14.9919356
  13.26438679 -14.53481121  14.78774523 -11.96744459 -12.46844499]


In [47]:
###save Catboost model###
pickle.dump(CB_model, open("CBoost.dat", "wb"))